In [5]:
import logging
import sys
import os
%cd ..
from bds.meel import approx_mc2, approx_mc2_core
from bds.rule import Rule
from bds.utils import bin_random , bin_array, randints, solutions_to_dict

from bds.bb import BranchAndBoundNaive, BranchAndBoundV1, BranchAndBoundV0
from logzero import logger
import gmpy2 as gmp
from gmpy2 import mpz , mpfr
from bds.sat.min_freq import construct_min_freq_program
from bds.sat.solver import construct_solver
from bds.gf2 import GF
from bds.rule import Rule
from bds.utils import mpz_set_bits
from tests.utils import assert_dict_allclose, assert_close_mpfr

from bds.bounds_utils import find_equivalence_classes
import numpy as np 

logger.setLevel(logging.DEBUG)

num_pts = 1000

ub = 0.8
lmbd = .1

delta = .8
eps = .8

show_progres = True
rand_seed = 1234

/u/50/ciaperm1/data/Desktop


## This notebook for testing purposes, it only helps with pen-and-paper computation for testing bounds

In [6]:
os.chdir("/u/50/ciaperm1/unix/Desktop/sampling-rashomon-decision-set-code")

## For end2end test 

In [7]:
rules = [
        Rule(
            id=1,
            name="rule-1",
            cardinality=1,
            # truthtable=np.array([0, 1, 0, 1, 0], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [1, 3])
        ),
        Rule(
            id=2,
            name="rule-2",
            cardinality=1,
            # truthtable=np.array([0, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [2, 4])
        ),
        Rule(
            id=3,
            name="rule-3",
            cardinality=1,
            # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [0, 2, 4])
        ),
    
        Rule(
            id=4,
            name="rule-4",
            cardinality=1,
            # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [0, 2, 4])
        ),
    
    
    ]


y = np.array([0, 1, 0, 0, 0], dtype=bool)

X = np.array( [[1,0,0], [0,1,0], [1,0,1], [0,1,0], [1,0,1]]  )

In [8]:
ub = float("inf")
lmbd = mpfr(lmbd)

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
first_elb, data_points2rules, equivalence_classes = find_equivalence_classes(y, rules)
bb.reset(first_elb)
node, not_captured = bb.queue.pop()


[D 230509 20:10:29 bb:74] calling BranchAndBoundV1 with ub=inf, lmbd=0.100000


5


In [9]:
data_points2rules

[[3, 4], [1], [2, 3, 4], [1], [2, 3, 4]]

In [11]:
data_points2rules[1]

[1]

In [12]:
example_rule_list = data_points2rules[1]
n = mpz_set_bits(gmp.mpz(), example_rule_list)
equivalence_classes[n].data_points


{1, 3}

In [8]:
equivalence_classes[n].total_positives


1

In [9]:
equivalence_classes[n].total_negatives


1

In [10]:
equivalence_classes[n].minority_mistakes

1

In [11]:
equivalence_classes

{mpz(24): <bds.bounds_utils.EquivalentPointClass at 0x7f67bf4f5c40>,
 mpz(2): <bds.bounds_utils.EquivalentPointClass at 0x7f67bf4f5880>,
 mpz(28): <bds.bounds_utils.EquivalentPointClass at 0x7f67bf4f56d0>}

In [12]:
equivalence_classes[mpz(24)].minority_mistakes

0

In [13]:
equivalence_classes[mpz(2)].minority_mistakes

1

In [14]:
equivalence_classes[mpz(28)].minority_mistakes

0

In [15]:
first_elb == -1/5

True

In [16]:
lmbd = 0.1
for ub in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]: 
    print( ((ub/lmbd) - 1) )

0.0
1.0
1.9999999999999996
3.0
4.0
4.999999999999999
5.999999999999999
7.0
8.0


In [17]:
lmbd = 0.1
for ub in [float("inf")]: 
    print( ((ub/lmbd) - 1) )

inf


In [112]:
lmbd = 0.1
ub = float("inf")
bb = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run( return_objective=True))
actual = solutions_to_dict(feasible_solutions)


[D 230509 19:54:28 bb:74] calling BranchAndBoundV0 with ub=inf, lmbd=0.1


In [113]:
actual

{(0, 1): mpfr('0.30000000000000004'),
 (0, 2): mpfr('0.69999999999999996'),
 (0, 3): mpfr('0.89999999999999991'),
 (0, 4): mpfr('0.89999999999999991'),
 (0, 1, 2): mpfr('0.80000000000000004'),
 (0, 1, 3): mpfr('1.0'),
 (0, 1, 4): mpfr('1.0'),
 (0, 2, 3): mpfr('1.0'),
 (0, 2, 4): mpfr('1.0'),
 (0, 3, 4): mpfr('1.0'),
 (0, 2, 3, 4): mpfr('1.0999999999999999'),
 (0, 1, 2, 3): mpfr('1.1000000000000001'),
 (0, 1, 2, 4): mpfr('1.1000000000000001'),
 (0, 1, 3, 4): mpfr('1.1000000000000001'),
 (0, 1, 2, 3, 4): mpfr('1.2000000000000002')}

In [44]:
expected = {(0, 1): mpfr('0.3'),
 (0, 2): mpfr('0.7'),
 (0, 3): mpfr('0.9'),
 (0, 4): mpfr('0.9'),
 (0, 1, 2): mpfr('0.8'),
 (0, 1, 3): mpfr('1.0'),
 (0, 1, 4): mpfr('1.0'),
 (0, 2, 3): mpfr('1.0'),
 (0, 2, 4): mpfr('1.0'),
 (0, 3, 4): mpfr('1.0'),
 (0, 2, 3, 4): mpfr('1.1'),
 (0, 1, 2, 3): mpfr('1.1'),
 (0, 1, 2, 4): mpfr('1.1'),
 (0, 1, 3, 4): mpfr('1.1'),
 (0, 1, 2, 3, 4): mpfr('1.2')}

In [46]:

for ub in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]: 
    print("ub " + str(ub))
    print( "threshold " + str(((ub/lmbd) - 1) )) 
    bb = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bb.run( return_objective=False))
    print(feasible_solutions)
    print(feasible_solutions)


[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.1, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.2, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.3, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.4, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.5, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.6, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.7, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.8, lmbd=0.1
[D 230509 18:41:07 bb:74] calling BranchAndBoundV0 with ub=0.9, lmbd=0.1


ub 0.1
threshold 0.0
[]
[]
ub 0.2
threshold 1.0
[]
[]
ub 0.3
threshold 1.9999999999999996
[]
[]
ub 0.4
threshold 3.0
[{0, 1}]
[{0, 1}]
ub 0.5
threshold 4.0
[{0, 1}]
[{0, 1}]
ub 0.6
threshold 4.999999999999999
[{0, 1}]
[{0, 1}]
ub 0.7
threshold 5.999999999999999
[{0, 1}, {0, 2}]
[{0, 1}, {0, 2}]
ub 0.8
threshold 7.0
[{0, 1}, {0, 2}, {0, 1, 2}]
[{0, 1}, {0, 2}, {0, 1, 2}]
ub 0.9
threshold 8.0
[{0, 1}, {0, 2}, {0, 3}, {0, 4}, {0, 1, 2}]
[{0, 1}, {0, 2}, {0, 3}, {0, 4}, {0, 1, 2}]


In [ ]:
expected = {(0, 1): mpfr('0.3'),
 (0, 2): mpfr('0.7'),
 (0, 3): mpfr('0.9'),
 (0, 4): mpfr('0.9'),
 (0, 1, 2): mpfr('0.8'),
 (0, 1, 3): mpfr('1.0'),
 (0, 1, 4): mpfr('1.0'),
 (0, 2, 3): mpfr('1.0'),
 (0, 2, 4): mpfr('1.0'),
 (0, 3, 4): mpfr('1.0'),
 (0, 2, 3, 4): mpfr('1.1'),
 (0, 1, 2, 3): mpfr('1.1'),
 (0, 1, 2, 4): mpfr('1.1'),
 (0, 1, 3, 4): mpfr('1.1'),
 (0, 1, 2, 3, 4): mpfr('1.2')}

In [ ]:
## Branch and boundV1 -- speed up 

In [50]:
lmbd = 0.1
ub = float("inf")

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(X_trn = X, return_objective=True))
actual = solutions_to_dict(feasible_solutions)



[D 230509 18:44:04 bb:74] calling BranchAndBoundV1 with ub=inf, lmbd=0.1


In [51]:
actual

{(0, 1): mpfr('0.30000000000000004'),
 (0, 2): mpfr('0.69999999999999996'),
 (0, 3): mpfr('0.89999999999999991'),
 (0, 4): mpfr('0.89999999999999991'),
 (0, 1, 2): mpfr('0.80000000000000004'),
 (0, 1, 3): mpfr('1.0'),
 (0, 1, 4): mpfr('1.0'),
 (0, 2, 3): mpfr('1.0'),
 (0, 2, 4): mpfr('1.0'),
 (0, 3, 4): mpfr('1.0'),
 (0, 2, 3, 4): mpfr('1.0999999999999999'),
 (0, 1, 2, 3): mpfr('1.1000000000000001'),
 (0, 1, 2, 4): mpfr('1.1000000000000001'),
 (0, 1, 3, 4): mpfr('1.1000000000000001'),
 (0, 1, 2, 3, 4): mpfr('1.2000000000000002')}

In [52]:
assert_dict_allclose(actual, expected)

In [9]:
2 / 5 + 1 / 5 +  0.1

0.7000000000000001

In [5]:
lmbd = 0.1
ub = float("inf")

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(X_trn = X, return_objective=True))
actual = solutions_to_dict(feasible_solutions)



[D 230509 18:51:55 bb:74] calling BranchAndBoundV1 with ub=inf, lmbd=0.1


rule Rule(id=1, name='rule-1', cardinality=1, truthtable=mpz(10))
parent rules {0}
parent_equivalent_lower_bound 0.2
equivalent lower bound after update 0.0
lower bound 0.30000000000000004
parent lower bound 0.0
rule Rule(id=2, name='rule-2', cardinality=1, truthtable=mpz(20))
parent rules {0}
parent_equivalent_lower_bound 0.2
equivalent lower bound after update 0.2
lower bound 0.5
parent lower bound 0.0
rule Rule(id=3, name='rule-3', cardinality=1, truthtable=mpz(21))
parent rules {0}
parent_equivalent_lower_bound 0.2
equivalent lower bound after update 0.2
lower bound 0.69999999999999996
parent lower bound 0.0
rule Rule(id=4, name='rule-4', cardinality=1, truthtable=mpz(21))
parent rules {0}
parent_equivalent_lower_bound 0.2
equivalent lower bound after update 0.2
lower bound 0.69999999999999996
parent lower bound 0.0
rule Rule(id=2, name='rule-2', cardinality=1, truthtable=mpz(20))
parent rules {0, 1}
parent_equivalent_lower_bound 0.0
equivalent lower bound after update 0.0
lower bo

#### It is correct, let us now consider a more interesting case with more equivalent bound , which actually prunes a lot 

In [14]:
rules = [
        Rule(
            id=1,
            name="rule-1",
            cardinality=1,
            # truthtable=np.array([0, 1, 0, 1, 0], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [1, 3, 5, 7])
        ),
        Rule(
            id=2,
            name="rule-2",
            cardinality=1,
            # truthtable=np.array([0, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [2, 4,8,9])
        ),
        Rule(
            id=3,
            name="rule-3",
            cardinality=1,
            # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [0, 2, 4,10])
        ),
    
        Rule(
            id=4,
            name="rule-4",
            cardinality=1,
            # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
            truthtable=mpz_set_bits(mpz(), [0, 2, 4, 11])
        )
      
    
    ]


y = np.array([0, 1, 0, 0, 1, 0 ,1, 1, 1, 0, 1, 1 ], dtype=bool)

X = np.array( [[1,0,0], [0,1,0], [1,0,1], [0,1,0], [1,0,1], [1,0,0], [0,1,0], [1,0,1], [0,1,0], [1,0,1], [0,1,0], [1,0,1]]  )

In [15]:
[y[i] for i in [1,3,5,7]]

[True, False, False, True]

In [67]:
[y[i] for i in range(len(y)) if i not in [1,3,5,7]]

[False, False, True, True, True, False, True, True]

In [77]:
for k in equivalence_classes: 
    print(k)

24
2
28
0
4
8
16


In [72]:
equivalence_classes[mpz(2)].minority_mistakes

2

In [108]:
len(data_points2rules)


12

In [79]:
data_points2rules[4]


[2, 3, 4]

In [80]:
data_points2rules[8]


[2]

In [81]:
data_points2rules[9]

[2]

In [89]:
mpzOutb = mpz_set_bits(gmp.mpz(),[2])

In [90]:
mpzOutb

mpz(4)

In [88]:
mpzOutb = mpz_set_bits(gmp.mpz(),[2])

In [92]:
equivalence_classes

{mpz(24): <bds.bounds_utils.EquivalentPointClass at 0x7f35d5e69d30>,
 mpz(2): <bds.bounds_utils.EquivalentPointClass at 0x7f35d5db9670>,
 mpz(28): <bds.bounds_utils.EquivalentPointClass at 0x7f35d5db9f10>,
 mpz(0): <bds.bounds_utils.EquivalentPointClass at 0x7f35d5dc50a0>,
 mpz(4): <bds.bounds_utils.EquivalentPointClass at 0x7f35d5cd5910>,
 mpz(8): <bds.bounds_utils.EquivalentPointClass at 0x7f35d5cd5a90>,
 mpz(16): <bds.bounds_utils.EquivalentPointClass at 0x7f35d98cf2e0>}

In [74]:
0.33 - 2 / 12

0.16333333333333336

In [60]:
assert len(y) == len(X)

In [61]:
first_elb, data_points2rules, equivalence_classes = find_equivalence_classes(X, y, rules)

In [62]:
first_elb

0.3333333333333333

In [75]:
2 / 12 + 0.1 


0.26666666666666666

In [17]:
lmbd = 0.1
ub = 0.7

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)



[D 230509 20:14:54 bb:74] calling BranchAndBoundV1 with ub=0.7, lmbd=0.1


12


In [18]:
actual

{(0, 1): mpfr('0.68333333333333335'),
 (0, 2): mpfr('0.68333333333333335'),
 (0, 3): mpfr('0.68333333333333335'),
 (0, 4): mpfr('0.68333333333333335'),
 (0, 3, 4): mpfr('0.69999999999999996')}

In [104]:
lmbd = 0.1
ub = 0.7

# the first iteration of the branch and bound
bb = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)


[D 230509 19:38:14 bb:74] calling BranchAndBoundV0 with ub=0.7, lmbd=0.1


In [105]:
actual

{(0, 1): mpfr('0.68333333333333335'),
 (0, 2): mpfr('0.68333333333333335'),
 (0, 3): mpfr('0.68333333333333335'),
 (0, 4): mpfr('0.68333333333333335'),
 (0, 3, 4): mpfr('0.69999999999999996')}